# Tarea 1 - Simultaneous Games

Alumnos:
José Ignacio Cáceres
Federico Azzato


Para la tarea se implementaron agentes inteligentes que aplican las técnicas de Fictitious Play y Regret Matching vistas en el curso, y se estudió el comportamiento de estos agentes interactuando entre sí, y junto a un agente random, en distintos juegos.

## Función de Evaluación

In [1]:
from base.agent import Agent
from base.game import SimultaneousGame
from typing import Callable
from typing import Iterable
import numpy as np

def EvaluateGame(game_name:str, game:SimultaneousGame, progress_display:Callable[[SimultaneousGame, Agent],None], agent_classes:Iterable[tuple[type[Agent],str]], iterations:int=10000):
    print('')
    print(f"Evaluating agents { ' vs '.join([agent_name for _, agent_name in agent_classes]) } in {game_name} game for {iterations} iterations")
    agents = {}
    game.reset()
    for i, agent in enumerate(game.agents):
        agents[agent] = agent_classes[i][0](game=game, agent=agent)

    for i in range(iterations):
        actions = dict(map(lambda agent: (agent, agents[agent].action()), game.agents))
        game.step(actions)
        if (i+1)%(iterations/10) == 0:
            print(f"Progress: {i+1}/{iterations} - ",
                dict(map(lambda agent: (agent, progress_display(game, agents[agent])), game.agents)))

    print("Final policy: ", dict(map(lambda agent: (agent, agents[agent].policy()), game.agents)))
    print('')

In [2]:
from agents.fictitiousplay import FictitiousPlay
from agents.regretmatching import RegretMatching
from agents.random_agent import RandomAgent

def EvaluateAll(game_name:str, game:SimultaneousGame, progress_display:Callable[[SimultaneousGame, Agent],None]=None, iterations:int=10000):
    smart_agent_classes = [
        (FictitiousPlay, 'FP'),
        (RegretMatching, 'RM')
        ]
    agent_combinations = []

    if progress_display is None:
        progress_display = lambda _, agent: agent.policy()

    for i in range(len(smart_agent_classes)):
        agent_combinations.append([smart_agent_classes[i], (RandomAgent, 'RA')])
        for j in range(i+1):
            agent_combinations.append([smart_agent_classes[i], smart_agent_classes[j]])

    print(f"Started Evaluation of {game_name} game")

    for agent_classes in agent_combinations:
        EvaluateGame(game_name, game, progress_display, agent_classes, iterations)

    print(f"Completed Evaluation of {game_name} game")

## Matching Pennies

In [3]:
from games.mp import MP

EvaluateAll('Matching Pennies', MP(), iterations=2000)

Started Evaluation of Matching Pennies game

Evaluating agents FP vs RA in Matching Pennies game for 2000 iterations
Progress: 200/2000 -  {'agent_0': array([0.98058252, 0.01941748]), 'agent_1': array([0.5, 0.5])}
Progress: 400/2000 -  {'agent_0': array([0.93103448, 0.06896552]), 'agent_1': array([0.5, 0.5])}
Progress: 600/2000 -  {'agent_0': array([0.95379538, 0.04620462]), 'agent_1': array([0.5, 0.5])}
Progress: 800/2000 -  {'agent_0': array([0.96526055, 0.03473945]), 'agent_1': array([0.5, 0.5])}
Progress: 1000/2000 -  {'agent_0': array([0.972167, 0.027833]), 'agent_1': array([0.5, 0.5])}
Progress: 1200/2000 -  {'agent_0': array([0.97678275, 0.02321725]), 'agent_1': array([0.5, 0.5])}
Progress: 1400/2000 -  {'agent_0': array([0.89544808, 0.10455192]), 'agent_1': array([0.5, 0.5])}
Progress: 1600/2000 -  {'agent_0': array([0.87048568, 0.12951432]), 'agent_1': array([0.5, 0.5])}
Progress: 1800/2000 -  {'agent_0': array([0.88150609, 0.11849391]), 'agent_1': array([0.5, 0.5])}
Progress:

Este juego es bastante simple. Entrenando con 2000 iteraciones se observa que no hay un equilibrio. Sin embargo tanto agente como oponente terminan aprendiendo una política equivalente a la aleatoria.

## Rock Paper Scisors

In [4]:
from games.rps import RPS

EvaluateAll('Rock Paper Scisors', RPS(), iterations=2000)

Started Evaluation of Rock Paper Scisors game

Evaluating agents FP vs RA in Rock Paper Scisors game for 2000 iterations
Progress: 200/2000 -  {'agent_0': array([0.71162791, 0.02790698, 0.26046512]), 'agent_1': array([0.33333333, 0.33333333, 0.33333333])}
Progress: 400/2000 -  {'agent_0': array([0.81686747, 0.03614458, 0.14698795]), 'agent_1': array([0.33333333, 0.33333333, 0.33333333])}
Progress: 600/2000 -  {'agent_0': array([0.63739837, 0.26341463, 0.09918699]), 'agent_1': array([0.33333333, 0.33333333, 0.33333333])}
Progress: 800/2000 -  {'agent_0': array([0.55828221, 0.36687117, 0.07484663]), 'agent_1': array([0.33333333, 0.33333333, 0.33333333])}
Progress: 1000/2000 -  {'agent_0': array([0.58128079, 0.35862069, 0.06009852]), 'agent_1': array([0.33333333, 0.33333333, 0.33333333])}
Progress: 1200/2000 -  {'agent_0': array([0.54979424, 0.4       , 0.05020576]), 'agent_1': array([0.33333333, 0.33333333, 0.33333333])}
Progress: 1400/2000 -  {'agent_0': array([0.51590106, 0.4409894 , 0

Al igual que en matching pennies, el juego es bastante simple. Al entrenarlo con 2000 itraciones se puede observar que tampoco se llega a un equilibrio. La política aprendida por los ageentes también converge a la política aleatoria.

## Blotto

### S=6

In [5]:
from games.blotto import Blotto

EvaluateAll('Blotto 6', Blotto(S=6, N=3), lambda game, agent: game._moves[np.argmax(agent.policy())], iterations=10000)

Started Evaluation of Blotto 6 game

Evaluating agents FP vs RA in Blotto 6 game for 10000 iterations
Progress: 1000/10000 -  {'agent_0': [2, 2, 2], 'agent_1': [1, 1, 4]}
Progress: 2000/10000 -  {'agent_0': [2, 2, 2], 'agent_1': [1, 1, 4]}
Progress: 3000/10000 -  {'agent_0': [2, 2, 2], 'agent_1': [1, 1, 4]}
Progress: 4000/10000 -  {'agent_0': [2, 2, 2], 'agent_1': [1, 1, 4]}
Progress: 5000/10000 -  {'agent_0': [2, 2, 2], 'agent_1': [1, 1, 4]}
Progress: 6000/10000 -  {'agent_0': [2, 2, 2], 'agent_1': [1, 1, 4]}
Progress: 7000/10000 -  {'agent_0': [2, 2, 2], 'agent_1': [1, 1, 4]}
Progress: 8000/10000 -  {'agent_0': [2, 2, 2], 'agent_1': [1, 1, 4]}
Progress: 9000/10000 -  {'agent_0': [2, 2, 2], 'agent_1': [1, 1, 4]}
Progress: 10000/10000 -  {'agent_0': [2, 2, 2], 'agent_1': [1, 1, 4]}
Final policy:  {'agent_0': array([7.98164222e-04, 6.98393695e-04, 9.98503442e-01]), 'agent_1': array([0.33333333, 0.33333333, 0.33333333])}


Evaluating agents FP vs FP in Blotto 6 game for 10000 iterations


Este juego es un poco más interesante. En el progreso se muestra la jugada de mayor probabilidad según la política para mantener el formato de la salida simple.

Se dejó entrenar por 10000 episodios. Se observa que cuando juegan 2 agentes FP, se alcanza un equilibrio jugando `[2,2,2]` mientras que los agentes RM encuentran como jugadas óptimas `[1,2,3]` y `[2,2,2]`.

### Blotto 12

In [6]:
EvaluateAll('Blotto 12', Blotto(S=12, N=3), lambda game, agent: game._moves[np.argmax(agent.policy())], iterations=20000)

Started Evaluation of Blotto 12 game

Evaluating agents FP vs RA in Blotto 12 game for 20000 iterations
Progress: 2000/20000 -  {'agent_0': [3, 4, 5], 'agent_1': [1, 1, 10]}
Progress: 4000/20000 -  {'agent_0': [2, 5, 5], 'agent_1': [1, 1, 10]}
Progress: 6000/20000 -  {'agent_0': [2, 5, 5], 'agent_1': [1, 1, 10]}
Progress: 8000/20000 -  {'agent_0': [2, 5, 5], 'agent_1': [1, 1, 10]}
Progress: 10000/20000 -  {'agent_0': [2, 5, 5], 'agent_1': [1, 1, 10]}
Progress: 12000/20000 -  {'agent_0': [2, 5, 5], 'agent_1': [1, 1, 10]}
Progress: 14000/20000 -  {'agent_0': [2, 5, 5], 'agent_1': [1, 1, 10]}
Progress: 16000/20000 -  {'agent_0': [2, 5, 5], 'agent_1': [1, 1, 10]}
Progress: 18000/20000 -  {'agent_0': [2, 5, 5], 'agent_1': [1, 1, 10]}
Progress: 20000/20000 -  {'agent_0': [2, 5, 5], 'agent_1': [1, 1, 10]}
Final policy:  {'agent_0': array([3.49040140e-04, 2.49314385e-04, 1.99451508e-04, 1.49588631e-04,
       1.49588631e-04, 4.98628771e-05, 9.97257542e-05, 3.98903017e-04,
       7.84941411e-01

En esta ejecución se dejó entrenar por 20000 iteraciones, para permitir que tanto FP como RM converjan. Las distribuciones de probabilidades finales muestran diferencias marginales entre algunas acciones disponibles, por lo que es probable que dejando entrenar un número mayor de iteraciones haya algún movimiento hacia otra posición.